In [10]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install Textblob

In [12]:
from textblob import TextBlob

In [13]:
import pandas as pd

In [14]:
data_path = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/ProssedBluBCA.csv'

In [15]:
!pip3 install googletrans==3.1.0a0

In [16]:
# Importing library yang dibutuhkan
import googletrans
from googletrans import Translator

In [17]:
data = pd.read_csv(data_path)
data.head()

,content_normalisasi2
0,ok
1,mantap pisan
2,memuaskan
3,transfer pake biaya admin jd male top up
4,mudah prosesnya tpi gak acc


In [18]:
translator = Translator()
translations = {}
for column in data.columns:
    # Unique elements of the column
    unique_elements = data[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element).text
translations

{'ok': 'ok',
 'mantap pisan': 'very stable',
 'memuaskan': 'satisfying',
 'transfer pake biaya admin jd male top up': 'transfers require admin fees so male top up',
 'mudah prosesnya tpi gak acc': 'easy process but not acc',
 'sayangnya transfer kena biaya admin': 'Unfortunately transfers are subject to admin fees',
 'sekrang tranfer dr blue kena admindmalas pakai lgmasih atap kena admindkecuali bank ok': "now the transfer from blu is subject to admin, lazy to use it, it's still subject to admin, unless the bank is ok",
 'lemot': 'slow',
 'sangat² membantu': 'very helpful',
 'udah bagus segi administrasinya udah prose pendaftaran membuka rekening bintang developer mobile penggunaan aplikasi terbilang berat memakan resource memory rendering tampilan istilahnya animasi design appsnya memukau kedepannya diharapkan': "It's already good in terms of administration, the registration process for opening an account, Bintang Mobile Developer, application usage is quite heavy, consuming memory re

In [19]:
data.replace(translations, inplace = True)
data

,content_normalisasi2
0,ok
1,very stable
2,satisfying
3,transfers require admin fees so male top up
4,easy process but not acc
...,...
4534,in
4535,I don't use a Blu BCA card but it's cool
4536,"What do you do, boss, if you already have an a..."
4537,"The bonus is really cool, the interest is free..."


In [20]:
data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTranslated.xlsx', index=False)

In [21]:
#SPLIT DATA
import pandas as pd

dataset = pd.read_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTranslated.xlsx')

# Hitung jumlah baris dalam dataset
num_rows = len(dataset)

# Hitung jumlah baris yang akan digunakan sebagai pelatihan (disini 50%)
train_size = int(0.5 * num_rows)

# Bagi dataset menjadi subset pelatihan dan pengujian
train_data = dataset[:train_size]
test_data = dataset[train_size:]

# Simpan subset pelatihan ke dalam file xlsx
train_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTrainEnglish.xlsx', index=False)

# Simpan subset pengujian ke dalam file xlsx
test_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTestEnglish.xlsx', index=False)


In [22]:
!pip install Textblob

In [23]:
from textblob import TextBlob

In [24]:
train_data

,content_normalisasi2
0,ok
1,very stable
2,satisfying
3,transfers require admin fees so male top up
4,easy process but not acc
...,...
2264,"It's good, it's slow, sorry, most of the anima..."
2265,"comfortable and safe, easy, always successful"
2266,x loan application rejected even though the ba...
2267,It's a shame it doesn't show QR Pay on Indomar...


In [25]:
data_content = list(train_data['content_normalisasi2'])
polaritas = 0

status = []
total_positif = total_negatif = total = 0

for i, tweet in enumerate(data_content):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

Hasil Analisis Data:
Positif = 1206
Negatif = 1063

Total Data : 2269


In [26]:
status = pd.DataFrame({'Label': status})
train_data['Label'] = status
train_data

<ipython-input-26-ddb1f5b9bf00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Label'] = status


,content_normalisasi2,Label
0,ok,Positif
1,very stable,Positif
2,satisfying,Positif
3,transfers require admin fees so male top up,Positif
4,easy process but not acc,Positif
...,...,...
2264,"It's good, it's slow, sorry, most of the anima...",Positif
2265,"comfortable and safe, easy, always successful",Positif
2266,x loan application rejected even though the ba...,Negatif
2267,It's a shame it doesn't show QR Pay on Indomar...,Negatif


In [27]:
# Memisahkan data berdasarkan label
positif_data = train_data[train_data['Label'] == 'Positif']
negatif_data = train_data[train_data['Label'] == 'Negatif']

In [28]:

# Menentukan jumlah sampel minimum di 2 kelas kelas
min_samples = min(total_positif, total_negatif)

In [29]:
from sklearn.utils import resample

In [30]:
# Melakukan penyeimbangan data dengan undersampling
positif_data_resampled = resample(positif_data, replace=False, n_samples=min_samples, random_state=42)
negatif_data_resampled = resample(negatif_data, replace=False, n_samples=min_samples, random_state=42)

In [31]:
# Menggabungkan kembali data yang telah disusun ulang
balanced_data = pd.concat([positif_data_resampled, negatif_data_resampled])

In [32]:
data_content_balanced = list(balanced_data['content_normalisasi2'])
polaritas_balanced = 0

status_balanced = []
total_positif_balanced = total_negatif_balanced = total_balanced = 0

for i, tweet in enumerate(data_content_balanced):
    analysis = TextBlob(tweet)
    polaritas_balanced += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif_balanced += 1
        status_balanced.append('Positif')
    else:
        total_negatif_balanced += 1
        status_balanced.append('Negatif')

    total_balanced += 1

print(f'Hasil Analisis Data Setelah Penyeimbangan:\nPositif = {total_positif_balanced}\nNegatif = {total_negatif_balanced}')
print(f'\nTotal Data Setelah Penyeimbangan: {total_balanced}')

Hasil Analisis Data Setelah Penyeimbangan:
Positif = 1063
Negatif = 1063

Total Data Setelah Penyeimbangan: 2126


In [33]:
train_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTrainEngLabeled.xlsx',index=False)

In [34]:
balanced_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTrainEngLabeledBalanced.xlsx', index=False)

In [ ]:
import pandas as pd
from googletrans import Translator

def translate_to_indonesian(text):
    translator = Translator()
    result = translator.translate(text, src='en', dest='id')
    return result.text

def translate_excel_column(input_file, output_file, sheet_name, column_name):
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df[column_name] = df[column_name].apply(translate_to_indonesian)

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    print("Translation complete. Translated data saved to a new Excel file!")

# Example usage
input_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTrainEngLabeledBalanced.xlsx'
output_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTrainIdnLabeledBalanced.xlsx'
sheet_name = 'Sheet1'  # Change to your sheet name
column_name = 'content_normalisasi2'  # Change to the name of the column you want to translate

translate_excel_column(input_file, output_file, sheet_name, column_name)

In [ ]:
import pandas as pd
from googletrans import Translator

def translate_to_indonesian(text):
    translator = Translator()
    result = translator.translate(text, src='en', dest='id')
    return result.text

def translate_excel_column(input_file, output_file, sheet_name, column_name):
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df[column_name] = df[column_name].apply(translate_to_indonesian)

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    print("Translation complete. Translated data saved to a new Excel file!")

# Example usage
input_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTestEnglish.xlsx'
output_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/BluBCA_DataTestIndo.xlsx'
sheet_name = 'Sheet1'  # Change to your sheet name
column_name = 'content_normalisasi2'  # Change to the name of the column you want to translate

translate_excel_column(input_file, output_file, sheet_name, column_name)